# Parquet to Dask
load a parquet dataset into as dask cluster

Once the client connects to the dask cluster we can check for package consistency by calling `dask_client.get_versions(check=True)`.  Scheduler and worker
nodes are guaranteed to have the same package versions in `mlrun` since mismatches would likely cause serialization failures.  The client (in this notebook or in some other process) may have different versions and this mismatch is **less likely** to cause issues.  The following installs can be performed to ensure this mismatch becomes **unlikely** (as of this writing). **This may need to be done periodically as the dockerfiles refresh, unless you specifiy an image tag.**

    !python -m pip install --upgrade pip
    !python -m pip uninstall -y cmake
    !python -m pip install cmake
    !python -m pip install scikit-build

    !python -m pip install lz4
    !python -m pip install git+https://github.com/Blosc/python-blosc.git@v1.8.3
    !python -m pip install cloudpickle==1.2.2

    !python -m pip install numpy==1.17.4

## clean up
delete stale pods

In [1]:
# !mlrun clean -p -r

In [2]:
import mlrun

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [3]:
# replace with your app node ip
mlrun.mlconf.remote_host = '18.223.249.136' 

## parameters


In [4]:
DESCRIPTION        = 'load parquet dataset into a dask cluster'       

IMAGE              = 'yjbds/mlrun-dask:dev'                           # custom image
JOB_KIND           = 'dask'                                           

TASK_NAME          = 'user-task-parq-to-dask'                         # whatever

ARTIFACTS_PATH     = '/User/repos/demos/dask/artifacts'
YAML_PATH          = '/User/repos/demos/dask/yaml'

DATA_PATH          = '/User/repos/demos/dask/dataset/partitions'
DOWNSAMPLE         = 0.01
PARTITION_COLS     = ['Year', 'Month']
PARTITION_COLS     = None

DASK_SHARDS        = 8
DASK_THREADS_PER   = 8
MEMORY_LIMIT       = '5GB'

DASK_KEY           = 'airlines'

## load and configure function

In [5]:
FUNC_PY   = '/User/repos/demos/dask/code/parquet-to-dask.py'
FUNC_YAML = '/User/repos/demos/dask/yaml/parquet-to-dask.yaml'

HANDLER   = 'parquet_to_dask'

#### some of the following cells can be commented out if this notebook has already been run.

In [6]:
# load function from a local Python file
parq2dask = mlrun.new_function(command=FUNC_PY, 
                               image=IMAGE,
                               kind=JOB_KIND)

parq2dask.spec.remote = True
parq2dask.spec.replicas = DASK_SHARDS 
parq2dask.spec.max_replicas = DASK_SHARDS
parq2dask.spec.service_type = 'NodePort'
parq2dask.spec.image_pull_policy = 'Always'
parq2dask.spec.build.image = IMAGE

parq2dask.export(FUNC_YAML)
# parq2dask = mlrun.import_function(FUNC_YAML)

parq2dask.apply(mlrun.mount_v3io())
parq2dask.deploy()

# create and run the task
parq_to_dask_task = mlrun.NewTask(
    TASK_NAME, 
    handler=HANDLER,  
    params={
        'parquet_url'      : DATA_PATH,
        'sample'           : DOWNSAMPLE,
        'shards'           : DASK_SHARDS,
        'threads_per'      : DASK_THREADS_PER,
        'memory_limit'     : MEMORY_LIMIT,
        'dask_key'         : DASK_KEY,
        'target_path'      : ARTIFACTS_PATH})

# run
rn = parq2dask.run(parq_to_dask_task)

[mlrun] 2020-02-13 06:28:00,162 function spec saved to path: /User/repos/demos/dask/yaml/parquet-to-dask.yaml
[mlrun] 2020-02-13 06:28:00,526 starting remote build, image: yjbds/mlrun-dask:dev
[mlrun] 2020-02-13 06:28:00,581 starting run user-task-parq-to-dask uid=26e89f78683d455694c7a8148553f186  -> http://mlrun-api:8080
[mlrun] 2020-02-13 06:28:02,058 saving function: parquet-to-dask, tag: latest
[mlrun] 2020-02-13 06:28:09,993 using remote dask scheduler (mlrun-parquet-to-dask-3fd6b276-a) at: 18.223.249.136:30640
[mlrun] 2020-02-13 06:28:09,993 remote dashboard (node) port: 18.223.249.136:31570


/conda/lib/python3.6/site-packages/distributed/client.py:1071: VersionMismatchWarning: Mismatched versions found

lz4
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 3.0.2   |
| scheduler | 2.2.1   |
+-----------+---------+

msgpack
+-----------+---------+
|           | version |
+-----------+---------+
| client    | 0.6.1   |
| scheduler | 0.6.2   |
+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


[mlrun] 2020-02-13 06:28:09,994 found cluster...
[mlrun] 2020-02-13 06:28:09,994 <Client: 'tcp://10.233.64.49:8786' processes=0 threads=0, memory=0 B>
[mlrun] 2020-02-13 06:30:26,606 column header ['Year' 'Month' 'DayofMonth' 'DayOfWeek' 'CRSDepTime' 'UniqueCarrier'
 'ArrDelay' 'Origin' 'Dest' 'Distance']
[mlrun] 2020-02-13 06:30:27,324 log artifact scheduler at /User/repos/demos/dask/artifacts/scheduler.json, size: None, db: Y

[mlrun] 2020-02-13 06:30:27,357 run ended with state 


/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/mlrun/render.py:324: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...53f186,0,Feb 13 06:28:02,completed,user-task-parq-to-dask,kind=daskowner=adminhost=jupyter-1-56795c755c-bss6b,,parquet_url=/User/repos/demos/dask/dataset/partitionssample=0.01shards=8threads_per=8memory_limit=5GBdask_key=airlinestarget_path=/User/repos/demos/dask/artifacts,,scheduler


to track results use .show() or .logs() or in CLI: 
!mlrun get run 26e89f78683d455694c7a8148553f186  , !mlrun logs 26e89f78683d455694c7a8148553f186 
[mlrun] 2020-02-13 06:30:27,415 run executed, status=completed


### the scheduler artifact can be shared

In [7]:
print(rn.metadata.to_yaml())

uid: 26e89f78683d455694c7a8148553f186
name: user-task-parq-to-dask
project: ''
labels:
  kind: dask
  owner: admin
  host: jupyter-1-56795c755c-bss6b
iteration: 0



In [8]:
rn.metadata.uid

'26e89f78683d455694c7a8148553f186'

In [9]:
rn.outputs['scheduler']

'/User/repos/demos/dask/artifacts/scheduler.json'

#### What's the scheduler address?

In [10]:
import json
json.load(open(rn.outputs['scheduler']))

{'type': 'Scheduler',
 'id': 'Scheduler-5ff43dd5-f780-4432-bc1d-1ef63f46deec',
 'address': 'tcp://10.233.64.49:8786',
 'services': {},
 'workers': {'tcp://10.233.64.52:41923': {'type': 'Worker',
   'id': 'tcp://10.233.64.52:41923',
   'host': '10.233.64.52',
   'resources': {},
   'local_directory': '/worker-nr6giqtt',
   'name': 'tcp://10.233.64.52:41923',
   'nthreads': 1,
   'memory_limit': 4109136640,
   'last_seen': 1581575427.151024,
   'services': {},
   'metrics': {'cpu': 4.0,
    'memory': 90488832,
    'time': 1581575427.150215,
    'read_bytes': 0.0,
    'write_bytes': 0.0,
    'num_fds': 22,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://10.233.64.52:36809'},
  'tcp://10.233.64.53:33822': {'type': 'Worker',
   'id': 'tcp://10.233.64.53:33822',
   'host': '10.233.64.53',
   'resources': {},
   'local_directory': '/worker-dso825q2',
   'name': 'tcp://10.233.64.5

### notebook cluster

Let's load the scheduler file into a cluster in this notebook using our MLRun artifact:

In [ ]:
# import dask
# import dask.dataframe as dd
# from dask.distributed import Client, LocalCluster

#client = Client(scheduler_file=rn.outputs['scheduler'])

#df = client.get_dataset(DASK_KEY)

### create a component 'on the fly' to summarise the table

The nice thing about having a dask cluster loaded with all your data is that you can write _quick and dirty_ jobs either in your notebook, a local file, or a github repo. Here we use a local file:

In [ ]:
# write up function in local directory
summ = mlrun.new_function(command='/User/repos/demos/dask/code/describe.py', 
                          image=IMAGE,
                          kind='job')

summ.spec.build.image = IMAGE

summ.export('/User/repos/demos/dask/yaml/describe.yaml')

summ.apply(mlrun.mount_v3io())

summ.deploy(skip_deployed=True, with_mlrun=False)

# create the task
summ_task = mlrun.NewTask(
    'user-task-my-sum', 
    handler='table_summary',  
    params={
        'dask_key'   :  DASK_KEY,
        'dask_client': 'scheduler.json',
        'target_path':  ARTIFACTS_PATH,
        'name'       : 'table-summary.csv',
        'key'        : 'table-summary'})

# run
rn2 = summ.run(summ_task)

rn2.outputs

### tests

In [ ]:
import pandas as pd
pd.read_csv(rn2.outputs['table-summary'])